## Install requirements

In [3]:
%%capture
import sys
import os
# installation of GOT (G eneralization O ver T axonomies) software package
!mkdir gotlib
if not os.listdir("gotlib"):
  !git clone https://github.com/dmitsf/GOT.git gotlib
sys.path.append('gotlib')


In [4]:
#get taxonomy

!python3 gotlib/got/taxonomies/taxonomy.py arxiv_category_taxonomy.csv

Python 


## Kaggle token

In [3]:
#from google.colab import files
#files.upload()

#!rm -r ~/.kaggle
#!mkdir ~/.kaggle
#!mv ./kaggle.json ~/.kaggle/
#!chmod 600 ~/.kaggle/kaggle.json

## Download Data

In [4]:
# Downloading data
#!pip install kaggle
#!kaggle datasets download -d Cornell-University/arxiv

#!mkdir Dataset
#!cp arxiv.zip Dataset/arxiv.zip
#!unzip -q Dataset/arxiv.zip -d Dataset
#!rm Dataset/arxiv.zip


In [5]:
import pandas as pd
dfs = pd.read_json('Dataset/arxiv-metadata-oai-snapshot.json', lines=True, chunksize = 1e3)

In [6]:
import re
import numpy as np

## Compute Relevance Matrix

In [7]:
from got.asts.ast import EASA


def clear_text(text, lowerize=True):

    pat = re.compile(r'[^A-Za-z0-9 \-\n\r.,;!?А-Яа-я]+')
    cleared_text = re.sub(pat, ' ', text)

    if lowerize:
        cleared_text = cleared_text.lower()

    tokens = cleared_text.split()
    return tokens


def make_substrings(tokens, k=4):

    for i in range(max(len(tokens) - k + 1, 1)):
        yield ' '.join(tokens[i:i + k])


def get_relevance_matrix(texts, strings):
    matrix = np.empty((0, len(strings)), float) 
    prepared_text_tokens = [clear_text(t) for t in texts]

    prepared_string_tokens = [clear_text(s) for s in strings]

    prepared_strings = [' '.join(t) for t in prepared_string_tokens]

    for text_tokens in prepared_text_tokens:
        ast = EASA(list(make_substrings(text_tokens)))
        row = np.array([ast.score(s) for s in prepared_strings])
        matrix = np.append(matrix, [row], axis=0)

    return matrix


def save_matrix(matrix, lb, ub):
    np.savetxt("relevance_matrix_{0}k-{1}k.txt".format(lb, ub), matrix)



with open("taxonomy_leaves.txt") as f:
    strings = [l.strip() for l in f.readlines()]


In [8]:
i = 0

In [ ]:
skip_first = 0
work_until = 1000

for data in dfs:
    if i < skip_first:
        i += 1
        continue
    print(data.index)
    df = data[['abstract']]

    sub_df = df.dropna(subset=['abstract'])
    abstracts = sub_df['abstract'].to_list()
    relevance_matrix = get_relevance_matrix(abstracts, strings)
    save_matrix(relevance_matrix, i, (i + 1))
    print("saved, shape: ", relevance_matrix.shape, " it: ", i)
    i += 1
    if i == work_until:
        break